In [5]:
# ! pip install numpy >> /dev/null
# ! pip install pandas >> /dev/null
# ! pip install opencv-python >> /dev/null
# ! pip install scipy >> /dev/null
# ! pip install torch >> /dev/null
# ! pip install matplotlib >> /dev/null
# ! pip install git+https://github.com/NVlabs/nvdiffrast >> /dev/null

# %pip install pyyaml

In [6]:
import torch
assert torch.cuda.is_available()


import os
import numpy as np
import pandas as pd
import cv2
import glob
import matplotlib.pyplot as plt

from tqdm import tqdm

##### Download camera paramethers

In [7]:
!pwd
import yaml

CAMERA = 0
downloaded_params = None

with open("camchain-calibration-equidistant4_camimu_dataset-calib-imu1.yaml") as stream:
    try:
        downloaded_params = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        raise Exception("Hello!")


/home/julfy/Documents/5th_term/cv/homework2


In [ ]:
INPUT_DIR = "./dataset-seq1/mav0/" # replace with your sequence
ORIGINAL_VIDEONAME = "output_orig.mp4"
UNDISTORTED_VIDEONAME = "output_undist.mp4"
RS_REMOVED_VIDEONAME = "output_nors.mp4"
STABILIZED_VIDEONAME = "output_stable_nors.mp4"


# camera params
FPS = 20

# camera intrinsics
pin_hole_model = downloaded_params[f"cam{CAMERA}"]["intrinsics"]
K = np.array([[pin_hole_model[0], 0, pin_hole_model[2]], 
              [0, pin_hole_model[1], pin_hole_model[3]],
              [0, 0, 1]])

# output camera intrinsics
K_new = K.copy()

# distortion params
D = downloaded_params[f"cam{CAMERA}"]["distortion_coeffs"]

# rotation from imu-to-camera
R_cam_imu = downloaded_params[f"cam{CAMERA}"]["T_cam_imu"]

# time delta between 2 sensor rows sensing 
dt_rs = 29.47 * 10 ** -6 # in seconds

# frame dimensions
W, H = 1280, 1024



pin_hole_model = [743.4286936207343, 743.5545205462922, 618.7186883884866, 506.7275058699658]
downloaded_params[f"cam{CAMERA}"]["T_cam_imu"] = [[-0.00596386151962109, -0.9999548067377452, 0.0074038393994521196, 0.05250867995679271], [-0.9999628101053462, 0.006009708608102947, 0.006185613038727172, 0.0077584026106338085], [-0.006229828408066805, -0.007366673911867888, -0.999953459593736, -0.04241050920541196], [0.0, 0.0, 0.0, 1.0]]


In [ ]:
def frames_to_video(frames, output_video, fps=20, isColor=False):
    """
    Create a video from a list of frames.

    :param frames: list of frames
    :param output_video: output video file name (e.g., 'output.mp4')
    :param fps: frames per second (default=20)
    """
    if len(frames) == 0:
        raise ValueError("No frames provided")

    # Use the first frame to get dimensions
    height, width, *channels = frames[0].shape

    # Define video writer (using mp4v codec)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(output_video, fourcc, fps, (width, height), isColor=isColor)

    for frame in tqdm(frames, desc="Writing frames to video: "):
        video.write(frame)
        

frames_paths = sorted(glob.glob(os.path.join(INPUT_DIR, "cam1/data/*.png")))
print(f"Number of frames in the video: {len(frames_paths)}")

frames = [cv2.imread(frame_path, cv2.IMREAD_GRAYSCALE) for frame_path in frames_paths]
frames_to_video(frames, ORIGINAL_VIDEONAME, FPS)

Number of frames in the video: 808


Writing frames to video: 100%|██████████| 808/808 [00:07<00:00, 103.29it/s]


### Homework 2

Given input video from camera with rolling shutter, implement and apply the following algorithms:

    - radial distortion correction (use cv2.fisheye module)
    - rolling shutter correction
    - video stabilization

Expected output of the homework:

    - Jupyter notebook with working code that we can run in real-time
    - plots of camera orientations in time for unstabilized and stabilized motion [plot individual x/y/z components in axis-angle representation]
    - plots of input/output meshgrids for the video frame, where angular velocity has the biggest norm
    - 4 videos stacked together with their frames side-by-side:
        - original
        - undistorted 
        - undistorted + rolling shutter corrected
        - undistorted + rolling shutter corrected + stabilized

Note:
Dataset used in homework provides two sources of camera orientations: IMU (Gyro) measurements of angular velocity and Motion Capture direct measurements of orientation. You should use IMU (Gyro) measurements and implement angular velocity integration. You can use Motion Capture direct measurements of orientation to verify that your integration is implemented correctly.

Rolling shutter correction algorithm is based on this paper: [Digital Video Stabilization and Rolling Shutter Correction using Gyroscopes](https://graphics.stanford.edu/papers/stabilization/).

Dataset: [Rolling-Shutter Visual-Inertial Odometry Dataset](https://cvg.cit.tum.de/data/datasets/rolling-shutter-dataset)

Calibrated camera params can be found [here](https://cdn3.vision.in.tum.de/rolling/calibration/camchain-calibration-equidistant4_camimu_dataset-calib-imu1.yaml)

![Camera Set-up](https://cvg.cit.tum.de/_media/data/datasets/rolling-shutter-dataset/sensor_axes_rgb.jpg?w=800&tok=fa3d4a)

### Undistort the video using **cv2 fisheye module**